**Matrix Computations** project (save part) \
*Lossy image compression using Singular Value Decompostion and Fast Fourier Transform*
## Nima Kelidari - 98108124 ##
**Matrix Computations course - Dr. razvan**

**Step0:** Import essential libraries

In [851]:
import cv2 
import numpy as np 
import imageio
import os
import shutil

**Step1:** Define compression values

In [852]:
#name = input('Input your file name (.png):')
name = 'pepper'

In [853]:
channels = ['b','g','r']
#name = 'spotify'
comp = 450
#threshhold -= 16
scale = 1.4
con_type = (2**15)-1


3.0000000000000013


**Step2:** Loading image and resize

In [854]:
img = cv2.imread(f'{name}.png')
img = cv2.resize(img, (0,0), fx=1/scale, fy=1/scale)
try:  
    os.mkdir(f'{name}')  
except OSError as error:  
    print(error)   

**Step3:** Define Save function for saving image as a folder

In [855]:
def save(name,img):

    config = channels.copy()
    config.append(con_type)
    config.append(scale)
    np.save(f'{name}/config.npy',config)

    def save_channel(th,color,u,s,v):
        uu = u[:,:th]
        vv = v[:th,:]
        ss = s[:th]

        mu = np.max(np.abs(uu))
        mv = np.max(np.abs(uu))

        im_u = np.array(np.round(((uu/mu)*con_type)+con_type),dtype = 'uint16')
        im_v = np.array(np.round(((vv/mv)*con_type)+con_type),dtype = 'uint16')

        np.save(f'{name}/max_{color}.npy',[mu,mv,th])
        np.save(f'{name}/s_{color}.npy',ss)
        imageio.imwrite(f'{name}/u_{color}.png',im_u)
        imageio.imwrite(f'{name}/v_{color}.png',im_v)

    for index,ch in enumerate(channels):
        u, s, v = np.linalg.svd(img[:,:,index], full_matrices=False)
        threshhold = len(s)
        for ind,i in enumerate(s):
            if i<comp:
                threshhold = ind
                break
        save_channel(threshhold,ch,u,s,v) 

**Step4:** Run *save* function, Convert folder to zip and delete the folder

In [856]:
save(name,img)
shutil.make_archive(name, 'zip', name)
shutil.rmtree(name)